<a href="https://colab.research.google.com/github/alezakuskin/Stark_ML/blob/Main/Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
#@title # Run this cell to get all dependencies and packages ready
from google.colab import output
from itertools import compress

import pandas as pd
import numpy as np
import xgboost

!git clone -b Main https://github.com/alezakuskin/Stark_ML
from Stark_ML.utils.terms import *
output.clear()

In [61]:
#@title Max width value
#@markdown ### By default your will get results on test linelist Example.xlsx.

#@markdown ###To get predictions on lines of your interest, please fill and upload (section 'Files' on the left panel) your own .xslx file and specify its name in the field below:

filename = 'Example.xlsx' #@param {type:"string"}
filename = 'Stark_ML/' + filename


#Importing pretrained model
model = xgboost.XGBRegressor()
model.load_model('Stark_ML/XGB_pretrained.json')


#Loading linelist
try:
    data_predictions = pd.read_excel(filename,
                     sheet_name='Sheet1',
                     usecols='A:BC'
                   )
except:
    data_predictions = pd.read_excel(filename[9:],
                     sheet_name='Sheet1',
                     usecols='A:BC'
                   )


#Data preprocessing
data_predictions['Term'] = term_to_number(data_predictions['Term'])
data_predictions['Term.1'] = term_to_number(data_predictions['Term.1'])


#Getting predictions
epsilon = 1e-3
preds = model.predict(data_predictions.drop(columns=['Element', 'Wavelength', 'Z number', 'w (A)', 'd (A)']))
preds = (np.exp(preds) - 1) * epsilon


#building output file
columns = ['Element', 'Wavelength', 'T', 'w (A)']
#@markdown 

#@markdown ###Select additional transition parameters you would like to include in output file
Element_symbol = True #@param {type: 'boolean'}
Wavelength = True #@param {type: 'boolean'}
Temperature = True #@param {type: 'boolean'}

results = pd.DataFrame(columns = list(compress(columns, [Element_symbol, Wavelength, Temperature, True])))
results['w (A)'] = preds
for i in results.columns[:-1]:
    results[i] = data_predictions[i]
results.to_csv(filename[9:-5] + '.csv', index = False)

[14:27:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


## Congratulations! If the previous cell finished execution without errors, you can now download <filename.csv> file with predicted values of Stark broadening parameter.

### For more details refer to 'paper' or contact us: ale-zakuskin@laser.chem.msu.ru